## Setup

In [1]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2015, day=15)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [2]:
# Import performance checking utility
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_example, time_solution

## Part a

### Iterative approach
I will start off by iterating over all 176,851 possible distributions of the four ingredients over 100 teaspoons.


In [3]:
# Imports
from typing import TYPE_CHECKING

import numpy as np

if TYPE_CHECKING:
    from collections.abc import Generator

In [ ]:
# Functions
def compositions_small(
    total: int, n: int
) -> Generator[tuple[int, int] | tuple[int, int, int] | tuple[int, int, int, int]]:
    """Yield pairs of 4-tuples of natural numbers summing to total.

    Note that this is about 10 times faster than a general solution using recursion or itertools.
    """
    if n == 2:
        for a in range(total + 1):
            yield (a, total - a)
    elif n == 4:
        for a in range(total + 1):
            for b in range(total - a + 1):
                for c in range(total - a - b + 1):
                    d = total - a - b - c
                    yield (a, b, c, d)
    else:
        msg = "n must be 2 or 4"
        raise ValueError(msg)


def parse_input(input_data: str) -> tuple[np.ndarray, np.ndarray]:
    """Parse the input data into a numpy array of ingredient properties and calories."""
    ingredients = np.array(
        [[int(part[-2:]) for part in line.split(",")] for line in input_data.splitlines()], dtype=np.int8
    )

    # Split off calories as they are only needed to check the contraint for part B
    props, calories = ingredients[:, :-1], ingredients[:, -1]

    return props, calories


def calc_score(
    tsp_counts: np.ndarray | tuple[int, ...], props: np.ndarray, calories: np.ndarray, *, target_calories: int | None
) -> int:
    """Calculate the cookie score for a given distribution of teaspoons."""
    totals = props.T @ tsp_counts

    # Enforce non-negative totals and calorie constraint
    if any(totals < 0) or (target_calories is not None and (calories @ tsp_counts) != target_calories):
        return 0

    # The score is product of all properties
    return int(np.prod(totals))


def solve_iterative(input_data: str, *, tsp: int = 100, target_calories: int | None = None) -> int:
    """Find the maximum cookie score using by iterating over all possible teaspoon combinations."""
    props, calories = parse_input(input_data)
    return max(
        calc_score(tsp_counts, props, calories, target_calories=target_calories)
        for tsp_counts in compositions_small(tsp, len(props))
    )

In [8]:
# Correctness check
check_example(solve_iterative, example, "a")

solve_iterative found answer 62842880, which is the correct solution for part A!


In [9]:
# Performance check
time_a_iterative = time_solution(solve_iterative, input_data, iterations=5, runs=3, time_unit="s")

solve_iterative takes 0.42 s


### Local search approach
The brute-force approach works, but it takes quite a while (0.4 s). I suspect that the distribution of scores is fairly continuous over the teaspoon distribution space, so a local search should be able to find a good solution much faster.

I'll start with an even distribution and iteratively improve it by transferring one teaspoon at a time until no further improvement is found. This process is repeated for random starting points to escape local maxima.

In [98]:
def hill_climb(
    start_tsp_count: np.ndarray,
    props: np.ndarray,
    calories: np.ndarray,
    *,
    target_calories: int | None,
    show_results: bool = False,
) -> int:
    """A single hill-climb that transfers one teaspoon at a time until no improvement."""
    current_tsp_count = start_tsp_count.copy()
    current_score = calc_score(current_tsp_count, props, calories, target_calories)
    improved = True

    m = len(props)

    while improved:
        # Check all single-teaspoon moves (remove 1 from i, add 1 to j) for improvement
        for i in range(m):
            improved = False
            if current_tsp_count[i] == 0:
                # This ingredient has no teaspoons to give
                continue

            for j in range(m):
                if i == j:
                    # No point transferring to self
                    continue

                candidate_tsp_count = current_tsp_count.copy()

                # Transfer one teaspoon from ingredient i to ingredient j
                candidate_tsp_count[i] -= 1
                candidate_tsp_count[j] += 1

                # Evaluate candidate
                candidate_score = calc_score(candidate_tsp_count, props, calories, target_calories)
                if candidate_score > current_score:
                    # Update current solution
                    current_score = candidate_score
                    current_tsp_count = candidate_tsp_count
                    improved = True

                    # Break to restart search from improved solution
                    break

            if improved:
                break

    if show_results:
        print(f"Hill-climb from {start_tsp_count} to {current_tsp_count} found score {current_score}")

    return current_score


def solve_local_search(
    input_data: str,
    *,
    tsp: int = 100,
    target_calories: int | None = None,
    restarts: int | None = None,
    dirichlet_alpha: float = 7.5,
    show_hill_climbs: bool = False,
) -> int:
    """Find the maximum cookie score using a local search with multiple random restarts.

    Args:
        input_data: The input data string.
        tsp: Total teaspoons to distribute among ingredients.
        target_calories: If set, only consider distributions with this total calorie count.
        restarts: Number of random restarts to perform. If None, set based on problem size.
        dirichlet_alpha: Parameter for Dirichlet distribution controlling randomness in restarts.
                         Defaults to 7.5, which was empirically found to work well for this problem.
        show_hill_climbs: If True, print details of each hill-climb.
    """
    props, calories = parse_input(input_data)
    m = len(props)

    best_score = 0

    # Initial solution: even distribution
    start_tsp_count = np.full(m, tsp // m, dtype=int)

    # Distribute remainder in case tsp is not divisible by the amount of ingredients
    start_tsp_count[: (tsp % m)] += 1

    # Perform hill-climb from initial solution
    best_score = max(
        best_score,
        hill_climb(start_tsp_count, props, calories, target_calories=target_calories, show_results=show_hill_climbs),
    )

    # Random restarts
    if restarts is None:
        # Set number of restarts based on problem size. Based on empirical testing.
        restarts = int(m * np.log1p(tsp // m))

    rng = np.random.default_rng(42)  # Fixed seed for reproducibility

    for _ in range(restarts):
        # Sample random distribution over ingredients from Dirichlet distribution
        # to avoid the default multinomial bias towards evenness
        p = rng.dirichlet(dirichlet_alpha * np.ones(m))

        # Random integer partition of tsp among ingredients
        rand_tsp_count = rng.multinomial(tsp, p)

        # Perform hill-climb from random start and update best score found
        best_score = max(
            best_score,
            hill_climb(rand_tsp_count, props, calories, target_calories=target_calories, show_results=show_hill_climbs),
        )

    return best_score

In [99]:
# Correctness check
check_example(solve_local_search, example, "a")

solve_local_search found answer 62842880, which is the correct solution for part A!


In [100]:
# Performance check
time_a_local_search = time_solution(solve_local_search, input_data)
print(f"This is {1000 * time_a_iterative / time_a_local_search:.0f}x faster than the iterative approach!")

solve_local_search takes 5.35 ms
This is 78x faster than the iterative approach!


Let's see how necessary the random restarts are:

In [76]:
solve_local_search(input_data, show_hill_climbs=True)

Hill-climb from [25 25 25 25] to [28 35 18 19] found score 13882464
Hill-climb from [24 30 18 28] to [28 35 18 19] found score 13882464
Hill-climb from [25 22 31 22] to [28 35 18 19] found score 13882464
Hill-climb from [31 17 25 27] to [31 17 25 27] found score 0
Hill-climb from [32 20 20 28] to [28 35 18 19] found score 13882464
Hill-climb from [27 27 21 25] to [28 35 18 19] found score 13882464
Hill-climb from [30 30 23 17] to [28 35 18 19] found score 13882464
Hill-climb from [38 23 23 16] to [28 35 18 19] found score 13882464
Hill-climb from [29 21 24 26] to [28 35 18 19] found score 13882464
Hill-climb from [26 34 23 17] to [28 35 18 19] found score 13882464
Hill-climb from [35 31 20 14] to [28 35 18 19] found score 13882464
Hill-climb from [21 25 21 33] to [28 35 18 19] found score 13882464
Hill-climb from [42 15 17 26] to [42 15 17 26] found score 0
Hill-climb from [30 11 29 30] to [30 11 29 30] found score 0


13882464

It turns out that starting from an even distribution already finds the optimal solution! So the random restarts are not necessary for this list of ingredients. If we just run the inital hill-climb from the even distribution, the speed improves even further:

In [77]:
time_a_local_search_no_repeats = time_solution(solve_local_search, input_data, restarts=0)
print(f"This is {1000 * time_a_iterative / time_a_local_search_no_repeats:.0f}x faster than the iterative approach!")

solve_local_search takes 0.58 ms
This is 714x faster than the iterative approach!


In [192]:
# Submit answer
puzzle.answer_a = solve_local_search(input_data)

## Part b

There is only a small difference between part a and part b: we need to enforce a calorie constraint of 500 calories. I've added the `target_calories` parameter to the solution functions to handle this, the changes in performance are shown below.

### Iterative approach
As we still need to iterate over all possible distributions, the performance is similar to part a:  

In [241]:
# Correctness check
check_example(solve_iterative, example, "b", target_calories=500)

solve_iterative found answer 57600000, which is the correct solution for part B!


True

In [95]:
# Performance check
time_b_iterative = time_solution(solve_iterative, input_data, target_calories=500, iterations=5, runs=3, time_unit="s")
print(f"This is {time_a_iterative / time_b_iterative:.2f}x faster than part a.")

solve_iterative takes 0.39 s
This is 1.06x faster than part a.


In [16]:
answer_b_iterative = solve_iterative(input_data, target_calories=500)
print(f"Guaranteed answer for part b: {answer_b_iterative}")

Guaranteed answer for part b: 11171160


### Local search approach
The calorie constraint makes the problem harder for the local search approach, as it makes many solutions invalid. This turns a continuous search space into one of a few discrete points, making it less likely that searching around a high-scoring point will find another high-scoring point that also satisfies the calorie constraint.

It's basically like turning the lights off in the search space and just gambling on a few random points to find a good solution. However, since a lot of the search space is invalid, the hill-climbs are performed much faster:

In [ ]:
# Performance check
time_b_local_search_init = time_solution(solve_local_search, input_data, target_calories=500)
print(f"This is {time_a_local_search / time_b_local_search_init:.2f}x faster than part a!")

solve_local_search takes 0.61 ms
This is 8.79x faster than part a!


However, note that the random restarts are now very much necessary to find the optimal solution, as we can see for the example input:

In [79]:
# Correctness check
check_example(solve_local_search, example, "b", target_calories=500, show_hill_climbs=True)

Hill-climb from [50 50] to [50 50] found score 0
Hill-climb from [40 60] to [40 60] found score 57600000
Hill-climb from [52 48] to [52 48] found score 0
Hill-climb from [42 58] to [42 58] found score 0
Hill-climb from [54 46] to [54 46] found score 0
Hill-climb from [48 52] to [48 52] found score 0
Hill-climb from [58 42] to [58 42] found score 0
Hill-climb from [76 24] to [76 24] found score 0
solve_local_search found answer 57600000, which is the correct solution for part B!


In fact, the local search approach hits invalid solutions spaces very often; e.g. only one out of 15 random restarts found a valid solution in the above run:

In [ ]:
answer_b_local_search_init = solve_local_search(input_data, target_calories=500, show_hill_climbs=True)
print(
    f"The local search approach found {answer_b_local_search_init}, after 15 hill-climbs, "
    f"but this is less than the actual answer {answer_b_iterative} guaranteed by checking all teaspoon distributions."
)

Hill-climb from [25 25 25 25] to [25 25 25 25] found score 9375000
Hill-climb from [24 30 18 28] to [24 30 18 28] found score 0
Hill-climb from [25 22 31 22] to [25 22 31 22] found score 0
Hill-climb from [31 17 25 27] to [31 17 25 27] found score 0
Hill-climb from [32 20 20 28] to [32 20 20 28] found score 0
Hill-climb from [27 27 21 25] to [27 27 21 25] found score 0
Hill-climb from [30 30 23 17] to [30 30 23 17] found score 0
Hill-climb from [38 23 23 16] to [38 23 23 16] found score 0
Hill-climb from [29 21 24 26] to [29 21 24 26] found score 0
Hill-climb from [26 34 23 17] to [26 34 23 17] found score 0
Hill-climb from [35 31 20 14] to [35 31 20 14] found score 0
Hill-climb from [21 25 21 33] to [21 25 21 33] found score 0
Hill-climb from [42 15 17 26] to [42 15 17 26] found score 0
Hill-climb from [30 11 29 30] to [30 11 29 30] found score 0
Hill-climb from [ 9 24 39 28] to [ 9 24 39 28] found score 0
Hill-climb from [35 26 18 21] to [35 26 18 21] found score 0
The local search a

We need to increase the amount of restarts by quite a lot, e.g. to 142, to find the optimal solution:

In [87]:
restarts = 142
if solve_local_search(input_data, target_calories=500, restarts=restarts) == answer_b_iterative:
    print(f"The local search approach found the correct answer after {restarts} hill-climbs!")

The local search approach found the correct answer after 142 hill-climbs!


Having to run so many restarts means that the performance is not as good as in part a, but it's still significantly faster than the iterative approach:

In [97]:
# Performance check
time_b_local_search = time_solution(solve_local_search, input_data, target_calories=500, restarts=142)
print(
    f"This is {time_b_local_search / time_a_local_search:.1f}x slower than part a, "
    f"but still {1000 * time_b_iterative / time_b_local_search:.1f}x faster than the iterative approach."
)

solve_local_search takes 6.29 ms
This is 1.2x slower than part a, but still 62.7x faster than the iterative approach.


In [85]:
# Submit answer
puzzle.answer_b = solve_local_search(input_data, target_calories=500, restarts=142)

That's the right answer!  You are one gold star closer to powering the weather machine.You have completed Day 15! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
